## ***0. Import libraries***

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
import jieba
import time
import numpy as np
import gensim
from gensim.utils import simple_preprocess

## ***1. Import sentiment dictionary***

In [ ]:
ANTUSD = {}
with open("dict/sentiment.csv", encoding='utf-8') as f:
    lines = [line.strip() for line in f.readlines()]
for line in lines:
    word = line.split(',')[0]
    score = line.split(',')[1]
    ANTUSD[word] = float(score)
# test
ANTUSD['良心']

In [ ]:
df = pd.read_csv("results/PP_leavehomesafe.csv", encoding = "utf-8")
df['content'] = df['thread_title'] + df['msg']
df['content'] = df['content'].astype(str)
data = df['content'].values.tolist()

In [ ]:
jieba.load_userdict("dict/sentimentDict.txt")

def is_chinese(uchar):
    if uchar >= '\u4e00' and uchar <= '\u9fa5':
        return True
    else:
        return False

def seg_depart(sentence):
    sentence_depart = jieba.cut(sentence.strip())
    outstr = ''
    for word in sentence_depart:
        if (len(word.strip()) > 0):
            outstr += word
            outstr += " "
    return outstr

result_fenci = []
for i in data:
    result_fenci.append([i, seg_depart(i)])
    
result_fenci = [i[1].split(' ')[:-1] for i in result_fenci]

In [ ]:
df['sentiment_tokenz'] = result_fenci

In [ ]:
def sent_to_words_space(sentences):
    for sent in sentences:
        sent = re.sub(r'[^\w\s]', '', sent)  # remove punctuations
        sent = sent.replace('_', '')
        yield (sent)


df['sentiment_tokenz'] = df['sentiment_tokenz'].astype(str)

msg_space = df.sentiment_tokenz.values.tolist()
msg_space_words = list(sent_to_words_space(msg_space))

print(msg_space_words[:1])

df['sentiment_tokenz'] = msg_space_words
df

In [ ]:
test_comment = df['sentiment_tokenz'].astype(str).values.tolist()

## ***2. Sentiment Labelling***

In [ ]:
def ANTUSD_approach(list_):
    ANTUSD_sentiment_list = []
    ANTUSD_post_token = []
    ANTUSD_neg_token = []
    ANTUSD_score = []
    for sentence in list_:
        score = 0
        temp_post = []
        temp_neg = []
        for token in sentence.split(' '):
            if token in ANTUSD:
                score = score + ANTUSD[token]
                if ANTUSD[token] > 0:
                    temp_post.append(token)
                else:
                    temp_neg.append(token)
        ANTUSD_post_token.append(temp_post)
        ANTUSD_neg_token.append(temp_neg)
        ANTUSD_score.append(score)
        if score > 0:
            ANTUSD_sentiment_list.append('positive')
        elif score == 0:
            ANTUSD_sentiment_list.append('neutral')     
        else:
            ANTUSD_sentiment_list.append('negative')
    return ANTUSD_sentiment_list, ANTUSD_post_token, ANTUSD_neg_token, ANTUSD_score 

In [ ]:
ANTUSD_sentiment_list, ANTUSD_post_token, ANTUSD_neg_token, ANTUSD_score = ANTUSD_approach(test_comment)

In [ ]:
df['sentiment'] = ANTUSD_sentiment_list
df['positive_terms'] = ANTUSD_post_token
df['negative_terms'] = ANTUSD_neg_token
df['sentiment_score']  = ANTUSD_score
df.head(5)

In [ ]:
df.pop('sentiment_tokenz')

In [ ]:
df.to_csv("results/stm_leavehomesafe.csv", encoding="utf-8", index=False)